In [1]:
pip install snowflake-sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import getpass
import pandas as pd
import plotly.express as px

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/snowflake/connector/options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.1), please install a version that adheres to: 'pyarrow<5.1.0,>=5.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [3]:
user = input("Snowflake username")
password = getpass.getpass("Snowflake password")
sf_engine = create_engine(
    URL(
        account="cw55877.east-us-2.azure",
        user=user,
        password=password,
        warehouse="ARM_WAREHOUSE",
        role=user,
        numpy=True,
    )
)

In [4]:
train_query = """
select *
from analytics.analytics.ml_ercot_all_features
where split = 'TRAIN'
order by delivery_datetime
"""

vali_query = """
select *
from analytics.analytics.ml_ercot_all_features
where split = 'VALI'
order by delivery_datetime
"""

test_query = """
select *
from analytics.analytics.ml_ercot_all_features
where split = 'TEST'
order by delivery_datetime
"""

df_train = pd.read_sql_query(train_query, con=sf_engine)
df_vali = pd.read_sql_query(vali_query, con=sf_engine)
df_test = pd.read_sql_query(test_query, con=sf_engine)

In [5]:
non_numeric_cols = [
    "DELIVERY_DATETIME", #	TIMESTAMP_TZ(9)
    # "DA_ENERGY_PRICE", #	FLOAT
    # "RT_ENERGY_PRICE", #	FLOAT
    # "DART_SPREAD", #	FLOAT
    # "DART_SPREAD_SIGN", #	FLOAT
    "DART_SPREAD_BIN", #	VARCHAR(16777216)
    "SPLIT", #	VARCHAR(7)
    # "ANCILLARY_PRICE_REGUP", #	FLOAT
    # "ANCILLARY_PRICE_REGDN", #	FLOAT
    # "ANCILLARY_PRICE_OFFNS", #	FLOAT
    # "ANCILLARY_PRICE_ONNS", #	FLOAT
    # "ANCILLARY_PRICE_RRSGN", #	FLOAT
    # "ANCILLARY_PRICE_RRSLD", #	FLOAT
    # "ANCILLARY_PRICE_RRSNC", #	FLOAT
    # "ANCILLARY_QUANTITY_REGUP", #	FLOAT
    # "ANCILLARY_QUANTITY_REGDN", #	FLOAT
    # "ANCILLARY_QUANTITY_OFFNS", #	FLOAT
    # "ANCILLARY_QUANTITY_ONNS", #	FLOAT
    # "ANCILLARY_QUANTITY_RRSGN", #	FLOAT
    # "ANCILLARY_QUANTITY_RRSLD", #	FLOAT
    # "ANCILLARY_QUANTITY_RRSNC", #	FLOAT
    "LOAD_ZONE_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "WEATHER_ZONE_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "ANCILLARY_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "STWPFANDWGRPP_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "WIND_PUBLISHED_DATETIME", #	TIMESTAMP_NTZ(9)
    "WEATHER_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "SOLAR_PUBLISHED_DATETIME", # TIMESTAMP_TZ(9)
    # "LOAD_ZONE_HOURDIFF", #	NUMBER(9,0)
    # "WEATHER_ZONE_HOURDIFF", #	NUMBER(9,0)
    # "ANCILLARY_HOURDIFF", #	NUMBER(9,0)
    # "STWPFANDWGRPP_HOURDIFF", #	NUMBER(9,0)
    # "WIND_HOURDIFF", #	NUMBER(9,0)
    # "WEATHER_HOURDIFF", #	NUMBER(9,0)
]
date_cols = [
    "DELIVERY_DATETIME", #	TIMESTAMP_TZ(9)
    "LOAD_ZONE_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "WEATHER_ZONE_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "ANCILLARY_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "STWPFANDWGRPP_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
    "WIND_PUBLISHED_DATETIME", #	TIMESTAMP_NTZ(9)
    "SOLAR_PUBLISHED_DATETIME", # TIMESTAMP_NTZ(9)
    "WEATHER_PUBLISHED_DATETIME", #	TIMESTAMP_TZ(9)
]

# convert numeric types
non_numeric_cols_lower = [col.lower() for col in non_numeric_cols]
numeric_cols = [col for col in df_train.columns if col not in non_numeric_cols_lower]
df_train[numeric_cols] = df_train[numeric_cols].astype('float')
df_vali[numeric_cols] = df_vali[numeric_cols].astype('float')
df_test[numeric_cols] = df_test[numeric_cols].astype('float')

# convert datetime types
date_cols_lower = [col.lower() for col in date_cols]
df_train[date_cols_lower] = df_train[date_cols_lower].astype(str)
df_vali[date_cols_lower] = df_vali[date_cols_lower].astype(str)
df_test[date_cols_lower] = df_test[date_cols_lower].astype(str)

assert df_train.dtypes.equals(df_vali.dtypes)
assert df_train.dtypes.equals(df_test.dtypes)

In [6]:
from azureml.core import Workspace, Dataset, Datastore

ws = Workspace.from_config()
blob_store = Datastore.get(ws, "babat")
ds_train = Dataset.Tabular.register_pandas_dataframe(
    df_train,
    blob_store,
    'ARM Training',
    description = 'ARM Training dataset pulled from snowflake')

ds_vali = Dataset.Tabular.register_pandas_dataframe(
    df_vali,
    blob_store,
    'ARM Validation',
    description = 'ARM Validation dataset pulled from snowflake')

ds_test = Dataset.Tabular.register_pandas_dataframe(
    df_test,
    blob_store,
    'ARM Testing',
    description = 'ARM Testing dataset pulled from snowflake')


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
